## 5 FUNÇÕES PARA FACILITAR O TRATAMENTO DE DADOS COM PYTHON - VICTOR TINTEL

In [3]:
# Importando os pacotes necessários

import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module='sklearn.feature_extraction.text')

In [4]:
# Carregando nosso conjunto de dados
df = pd.read_csv("itens.csv", sep = ';')
df.head()

,CLIENTE,ESTADO,CONSUMO,CATEGORIA
0,A1,SP,"Brinco, Anel, Pulseira, Colar",PREMIUM
1,A2,SP,"Pulseira, Colar, Gargantilha",TOP
2,A3,SP,"Brinco, Corrente",NORMAL
3,A4,PR,"Brinco, Anel, Bracelete, Relogio",PREMIUM
4,A5,PR,"Pulseira, Colar",TOP


In [7]:
# Verificando as dimensões do arquivo
df.shape

(186, 4)

In [9]:
# Verificando os tipos de variaveis
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 186 entries, 0 to 185
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   CLIENTE    186 non-null    object
 1   ESTADO     186 non-null    object
 2   CONSUMO    186 non-null    object
 3   CATEGORIA  186 non-null    object
dtypes: object(4)
memory usage: 5.9+ KB


In [11]:
# Fazendo um agrupamento por Estado e Categoria
tabela = df.groupby(["ESTADO", "CATEGORIA"]).size()
tabela

ESTADO  CATEGORIA
MG      NORMAL        7
        PREMIUM       7
        TOP           7
PR      NORMAL       16
        PREMIUM      16
        TOP          16
RJ      NORMAL        7
        PREMIUM       7
        TOP           7
SC      NORMAL       16
        PREMIUM      16
        TOP          16
SP      NORMAL       16
        PREMIUM      16
        TOP          16
dtype: int64

In [13]:
tabela1 = df.groupby(["ESTADO", "CATEGORIA"])
tabela1

### 1º Função unstack
<p>
unstack() transforma os níveis de índice (linhas) em colunas
<br>fill_value = 0  irá preencher onde for nulo como zero

In [15]:
# Criando nossa tabela utilizando o unstack
tabela_unstack = df.groupby(["ESTADO", "CATEGORIA"]).size().unstack(fill_value=0)
tabela_unstack

CATEGORIA,NORMAL,PREMIUM,TOP
ESTADO,,,
MG,7,7,7
PR,16,16,16
RJ,7,7,7
SC,16,16,16
SP,16,16,16


### 2º Função CountVectorizer

<br>O CountVectorizer é uma ferramenta da biblioteca sklearn.feature_extraction.text que transforma texto em uma matriz de contagem de palavras (também chamado de bag-of-words)


tokenizer=lambda x: x.split(", ")
<br>Essa função personalizada diz ao CountVectorizer exatamente como separar os itens da lista, usando a vírgula e um espaço como delimitador.



<br>Nós temos isso: "Brinco, Anel, Pulseira, Colar"
<br>E vamos transformar nisso: ["Brinco", "Anel", "Pulseira","Colar"]


Isso faz com que o CountVectorizer trate cada item como uma "palavra" única, permitindo gerar uma matriz com 1s e 0s indicando se aquele item aparece ou não em cada linha.

In [17]:
# A função lambda poderia ser trocada também por.. 
### def separar_itens(texto):
    ## return texto.split(", ")

vectorizer = CountVectorizer(tokenizer=lambda x: x.split(", "))
X_produtos = vectorizer.fit_transform(df["CONSUMO"])
X_produtos.toarray()

array([[1, 0, 1, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 1, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 1, 0],
       [1, 0, 1, ..., 0, 1, 0],
       [1, 1, 1, ..., 0, 0, 1]], dtype=int64)

In [19]:
# Aqui vamos converter o nosso vetor para um DataFrame
df_produtos = pd.DataFrame(X_produtos.toarray(), columns=vectorizer.get_feature_names_out())
df_produtos.head()

,anel,bracelete,brinco,colar,corrente,gargantilha,pingente,pulseira,relogio
0,1,0,1,1,0,0,0,1,0
1,0,0,0,1,0,1,0,1,0
2,0,0,1,0,1,0,0,0,0
3,1,1,1,0,0,0,0,0,1
4,0,0,0,1,0,0,0,1,0


In [21]:
# Agora vamos concatenar com nosso DataFrame
df = pd.concat([df[["CLIENTE", "ESTADO"]], df_produtos], axis=1)
df.head()

,CLIENTE,ESTADO,anel,bracelete,brinco,colar,corrente,gargantilha,pingente,pulseira,relogio
0,A1,SP,1,0,1,1,0,0,0,1,0
1,A2,SP,0,0,0,1,0,1,0,1,0
2,A3,SP,0,0,1,0,1,0,0,0,0
3,A4,PR,1,1,1,0,0,0,0,0,1
4,A5,PR,0,0,0,1,0,0,0,1,0


### 3º Função difference
<p>A função difference permite extrair as colunas de um dataframe com excessão de colunas que estão dentro da lista do método difference

In [23]:
# Pelo método tradicional fariamos da seguinte forma
colunas_produtos = ['anel', 'bracelete', 'brinco', 'colar', 'corrente', 'gargantilha', 'pingente', 'pulseira', 'relogio']
colunas_produtos

['anel',
 'bracelete',
 'brinco',
 'colar',
 'corrente',
 'gargantilha',
 'pingente',
 'pulseira',
 'relogio']

In [27]:
# Criando uma lista de colunas com o comando difference
# Vamos colocar aqui as colunas que não queremos, que no caso é CLIENTE E ESTADO.
colunas_produtos = df.columns.difference(['CLIENTE','ESTADO'])
colunas_produtos

Index(['anel', 'bracelete', 'brinco', 'colar', 'corrente', 'gargantilha',
       'pingente', 'pulseira', 'relogio'],
      dtype='object')

### 4º Função idxmax

<p> idxmax --> Para cada linha encontra o nome da coluna com o maior valor, ou seja, retorna o nome do produto mais consumido por estado
<br>to_frame --> Transforma uma série em um DataFrame e cria a coluna chamada PRODUTO_MAIS_CONSUMIDO

In [29]:
consumo_por_estado = df.groupby("ESTADO")[colunas_produtos].sum()
consumo_por_estado

,anel,bracelete,brinco,colar,corrente,gargantilha,pingente,pulseira,relogio
ESTADO,,,,,,,,,
MG,7,0,14,14,7,7,0,14,0
PR,16,26,32,16,16,0,16,16,16
RJ,14,7,14,14,0,0,0,14,7
SC,32,16,16,16,16,16,16,32,16
SP,16,0,32,32,16,16,0,32,0


In [31]:
# Verificando os produtos mais consumidos por estado
mais_consumido_por_estado = consumo_por_estado.idxmax(axis=1).to_frame(name="PRODUTO_MAIS_CONSUMIDO")
mais_consumido_por_estado

,PRODUTO_MAIS_CONSUMIDO
ESTADO,
MG,brinco
PR,brinco
RJ,anel
SC,anel
SP,brinco


In [33]:
# Verificando a quantidade do produto mais consumido por estado
consumo_total = consumo_por_estado.max(axis=1).to_frame(name="TOTAL_CONSUMO")
consumo_total

,TOTAL_CONSUMO
ESTADO,
MG,14
PR,32
RJ,14
SC,32
SP,32


In [35]:
# Visualizando o produto a quantidade total e a proporção de consumo por estado
resultado = pd.concat([mais_consumido_por_estado, consumo_total], axis=1)
resultado

,PRODUTO_MAIS_CONSUMIDO,TOTAL_CONSUMO
ESTADO,,
MG,brinco,14
PR,brinco,32
RJ,anel,14
SC,anel,32
SP,brinco,32


### 5º Função nlargest

<p>O método .nlargest() do pandas é usado para retornar os N maiores valores de uma série ou coluna de DataFrame, ordenados do maior para o menor.

In [37]:
# Visualizando o consumo por estado
consumo_por_estado.head()

,anel,bracelete,brinco,colar,corrente,gargantilha,pingente,pulseira,relogio
ESTADO,,,,,,,,,
MG,7,0,14,14,7,7,0,14,0
PR,16,26,32,16,16,0,16,16,16
RJ,14,7,14,14,0,0,0,14,7
SC,32,16,16,16,16,16,16,32,16
SP,16,0,32,32,16,16,0,32,0


In [39]:
# Verificando os 3 produtos mais consumidos por estado
top_3_por_estado = consumo_por_estado.apply(lambda linha: linha.nlargest(3).index.tolist(), axis=1)
top_3_por_estado

ESTADO
MG      [brinco, colar, pulseira]
PR      [brinco, bracelete, anel]
RJ          [anel, brinco, colar]
SC    [anel, pulseira, bracelete]
SP      [brinco, colar, pulseira]
dtype: object

In [41]:
# Gerando um DataFrame com os 3 produtos mais consumidos por estado

# Aqui estamos criando um DataFrame. O comando apply(pd.Series) ira transformar a lista que temos com 1 produto em cada coluna
top_3_df = top_3_por_estado.apply(pd.Series)

# Aqui estamos nomeando cada coluna
top_3_df.columns = ["1º Mais Consumido", "2º Mais Consumido", "3º Mais Consumido"]

# Aqui estamos obtendo a sigla do Estado e colocando na coluna chamada ESTADO
top_3_df["ESTADO"] = top_3_por_estado.index

# Aqui estamos definindo que a coluna ESTADO será o indice do DataFrame
top_3_df.set_index("ESTADO", inplace=True)

# Aqui estamos simplesmente exibindo o DataFrame
top_3_df

,1º Mais Consumido,2º Mais Consumido,3º Mais Consumido
ESTADO,,,
MG,brinco,colar,pulseira
PR,brinco,bracelete,anel
RJ,anel,brinco,colar
SC,anel,pulseira,bracelete
SP,brinco,colar,pulseira
